In [1]:
import picamera
import picamera.array

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import time
import numpy as np
import cv2
from PIL import Image
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
import cv2
import numpy as np
import os


Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [4]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)


In [5]:
def prepare_frame(frame):
    img = Image.fromarray(frame, 'RGB')
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array_extended = np.expand_dims(img_array, axis=0).astype('float32')
    processed = keras.applications.mobilenet.preprocess_input(img_array_extended)
    return processed

In [6]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
# model.save('my_model.h5')

# # Deletes the existing model
# del model  

# Returns a compiled model identical to the previous one
model = load_model('shape-detection.h5')

In [7]:
import paho.mqtt.client as mqtt
mqttc = mqtt.Client()
mqttc.connect("mqtt.cmmc.io", 1883, 60)
# mqttc.publish("CMMC/PLUG-001/$/command", "OFF")
# mqttc.publish("CMMC/PLUG-001/$/command", "ON")

mqttc2 = mqtt.Client()
mqttc2.connect("localhost", 1883, 60)
mqttc2.publish("KBML/class", "0")

In [8]:
# mqttc2.publish("lamp", "off")

In [9]:
# mqttc2.publish("lamp", "on")

In [10]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os

# from __future__ import print_function
from imutils.video import VideoStream
from imutils.video import FPS
from picamera.array import PiRGBArray
from picamera import PiCamera
import argparse
import imutils
import time
import cv2
from imutils.video import FPS
import datetime
import time
import numpy as np
import cv2
from PIL import Image

#image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# initialize the camera and stream
camera = PiCamera()
# camera.resolution = (320, 240)
camera.framerate = 32
camera.resolution = (224, 224)
camera.rotation = (270)
camera.start_preview()
# rawCapture = PiRGBArray(camera, size=(320, 240))
rawCapture = PiRGBArray(camera)
stream = camera.capture_continuous(rawCapture, format="bgr",use_video_port=True)

image_widget = widgets.Image()
display(text)
display(image_widget)


for (i, f) in enumerate(stream):
    frame = f.array
    image_widget.value =  bgr8_to_jpeg(frame)
    processed_image = prepare_frame(frame)  # prepare frame
    t1 = datetime.datetime.now()
    prediction = model.predict(processed_image)
    t2 = datetime.datetime.now()
    result = np.argmax(prediction)  # imagenet_utils.decode_predictions(prediction)
    text.value = "class=" + str(result)
    
    if str(result) == "0":
        mqttc2.publish("KBML/class", "0")      
    elif str(result) == "1":
        mqttc2.publish("KBML/class", "1") 
    elif str(result) == "2":
        mqttc2.publish("KBML/class", "2")
    elif str(result) == "3":
        mqttc2.publish("KBML/class", "3")
        
#         mqttc.publish("CMMC/PLUG-001/$/command", "OFF")
#         mqttc.publish("CMMC/PLUG-002/$/command", "OFF")    
    rawCapture.truncate(0)

Text(value='Hello World', description='String:', placeholder='Type something')

Image(value=b'')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-64a08c5a7a80>", line 49, in <module>
    prediction = model.predict(processed_image)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 1462, in predict
    callbacks=callbacks)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/backend.py", line 3292, in __call__
    run_metadata=self.run_metadata)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1458, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-pac

KeyboardInterrupt: 